# Data Cleaning

## Setup

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
path_raw = "../data/Fitabase Data 4.12.16-5.12.16/"

path_clean = "../data/cleaned/"

---

## Daily

### Daily Activity

In [61]:
daily = pd.read_csv(path_raw + "dailyActivity_merged.csv")
daily

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1503960366,4/12/2016,13162,8.500000,8.500000,0.0,1.88,0.55,6.06,0.00,25,13,328,728,1985
1,1503960366,4/13/2016,10735,6.970000,6.970000,0.0,1.57,0.69,4.71,0.00,21,19,217,776,1797
2,1503960366,4/14/2016,10460,6.740000,6.740000,0.0,2.44,0.40,3.91,0.00,30,11,181,1218,1776
3,1503960366,4/15/2016,9762,6.280000,6.280000,0.0,2.14,1.26,2.83,0.00,29,34,209,726,1745
4,1503960366,4/16/2016,12669,8.160000,8.160000,0.0,2.71,0.41,5.04,0.00,36,10,221,773,1863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,8877689391,5/8/2016,10686,8.110000,8.110000,0.0,1.08,0.20,6.80,0.00,17,4,245,1174,2847
936,8877689391,5/9/2016,20226,18.250000,18.250000,0.0,11.10,0.80,6.24,0.05,73,19,217,1131,3710
937,8877689391,5/10/2016,10733,8.150000,8.150000,0.0,1.35,0.46,6.28,0.00,18,11,224,1187,2832
938,8877689391,5/11/2016,21420,19.559999,19.559999,0.0,13.22,0.41,5.89,0.00,88,12,213,1127,3832


In [62]:
# Check for missing data
daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 940 entries, 0 to 939
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Id                        940 non-null    int64  
 1   ActivityDate              940 non-null    object 
 2   TotalSteps                940 non-null    int64  
 3   TotalDistance             940 non-null    float64
 4   TrackerDistance           940 non-null    float64
 5   LoggedActivitiesDistance  940 non-null    float64
 6   VeryActiveDistance        940 non-null    float64
 7   ModeratelyActiveDistance  940 non-null    float64
 8   LightActiveDistance       940 non-null    float64
 9   SedentaryActiveDistance   940 non-null    float64
 10  VeryActiveMinutes         940 non-null    int64  
 11  FairlyActiveMinutes       940 non-null    int64  
 12  LightlyActiveMinutes      940 non-null    int64  
 13  SedentaryMinutes          940 non-null    int64  
 14  Calories  

In [63]:
# Drop duplicates
daily = daily.drop_duplicates(["Id", "ActivityDate"])

### Create WoreDevice Feature

In [64]:
# Add feature indicating whether device was worn that day
daily["WoreDevice"] = ((daily.TotalSteps > 0) | (daily.Calories > 0))
daily[daily.WoreDevice == False]

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,WoreDevice
30,1503960366,5/12/2016,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1440,0,False
653,6290855005,5/10/2016,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1440,0,False
817,8253242879,4/30/2016,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1440,0,False
879,8583815059,5/12/2016,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1440,0,False


### Merge Sleep Data

In [65]:
# Load daily sleep data
daily_sleep = pd.read_csv(path_raw + "sleepDay_merged.csv")

# Convert date column to match
daily_sleep = daily_sleep.rename({"SleepDay": "ActivityDate"}, axis=1)
daily_sleep.ActivityDate = daily_sleep.ActivityDate.str.split(" ").str.get(0)

daily_sleep

,Id,ActivityDate,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016,1,327,346
1,1503960366,4/13/2016,2,384,407
2,1503960366,4/15/2016,1,412,442
3,1503960366,4/16/2016,2,340,367
4,1503960366,4/17/2016,1,700,712
...,...,...,...,...,...
408,8792009665,4/30/2016,1,343,360
409,8792009665,5/1/2016,1,503,527
410,8792009665,5/2/2016,1,415,423
411,8792009665,5/3/2016,1,516,545


In [66]:
# Check duplicate data
daily_sleep[daily_sleep.duplicated(["Id", "ActivityDate"], False)]

,Id,ActivityDate,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
160,4388161847,5/5/2016,1,471,495
161,4388161847,5/5/2016,1,471,495
222,4702921684,5/7/2016,1,520,543
223,4702921684,5/7/2016,1,520,543
379,8378563200,4/25/2016,1,388,402
380,8378563200,4/25/2016,1,388,402


In [67]:
# Drop duplicates and merge
daily_sleep = daily_sleep.drop_duplicates(["Id", "ActivityDate"])

# Merge sleep data
daily = pd.merge(daily, daily_sleep, on=["Id", "ActivityDate"], how="left")

daily

,Id,ActivityDate,TotalSteps,TotalDistance,TrackerDistance,LoggedActivitiesDistance,VeryActiveDistance,ModeratelyActiveDistance,LightActiveDistance,SedentaryActiveDistance,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories,WoreDevice,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,4/12/2016,13162,8.500000,8.500000,0.0,1.88,0.55,6.06,0.00,25,13,328,728,1985,True,1.0,327.0,346.0
1,1503960366,4/13/2016,10735,6.970000,6.970000,0.0,1.57,0.69,4.71,0.00,21,19,217,776,1797,True,2.0,384.0,407.0
2,1503960366,4/14/2016,10460,6.740000,6.740000,0.0,2.44,0.40,3.91,0.00,30,11,181,1218,1776,True,NaN,NaN,NaN
3,1503960366,4/15/2016,9762,6.280000,6.280000,0.0,2.14,1.26,2.83,0.00,29,34,209,726,1745,True,1.0,412.0,442.0
4,1503960366,4/16/2016,12669,8.160000,8.160000,0.0,2.71,0.41,5.04,0.00,36,10,221,773,1863,True,2.0,340.0,367.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,8877689391,5/8/2016,10686,8.110000,8.110000,0.0,1.08,0.20,6.80,0.00,17,4,245,1174,2847,True,NaN,NaN,NaN
936,8877689391,5/9/2016,20226,18.250000,18.250000,0.0,11.10,0.80,6.24,0.05,73,19,217,1131,3710,True,NaN,NaN,NaN
937,8877689391,5/10/2016,10733,8.150000,8.150000,0.0,1.35,0.46,6.28,0.00,18,11,224,1187,2832,True,NaN,NaN,NaN
938,8877689391,5/11/2016,21420,19.559999,19.559999,0.0,13.22,0.41,5.89,0.00,88,12,213,1127,3832,True,NaN,NaN,NaN


### Write CSV

In [70]:
# Write clean data
daily.to_csv(path_clean + "daily_clean.csv", index=False)

---

## Hourly

### Load data

In [113]:
hourly_fnames = [f for f in os.listdir(path_raw) if "hourly" in f.lower()]

hourly_dfs = [pd.read_csv(path_raw + f) for f in hourly_fnames]

for df in hourly_dfs:
    display(df.head())

,Id,ActivityHour,Calories
0,1503960366,4/12/2016 12:00:00 AM,81
1,1503960366,4/12/2016 1:00:00 AM,61
2,1503960366,4/12/2016 2:00:00 AM,59
3,1503960366,4/12/2016 3:00:00 AM,47
4,1503960366,4/12/2016 4:00:00 AM,48


,Id,ActivityHour,TotalIntensity,AverageIntensity
0,1503960366,4/12/2016 12:00:00 AM,20,0.333333
1,1503960366,4/12/2016 1:00:00 AM,8,0.133333
2,1503960366,4/12/2016 2:00:00 AM,7,0.116667
3,1503960366,4/12/2016 3:00:00 AM,0,0.000000
4,1503960366,4/12/2016 4:00:00 AM,0,0.000000


,Id,ActivityHour,StepTotal
0,1503960366,4/12/2016 12:00:00 AM,373
1,1503960366,4/12/2016 1:00:00 AM,160
2,1503960366,4/12/2016 2:00:00 AM,151
3,1503960366,4/12/2016 3:00:00 AM,0
4,1503960366,4/12/2016 4:00:00 AM,0


### Drop Duplicates

In [115]:
for i in range(len(hourly_dfs)):
    print(hourly_fnames[i])
    print(f"Before: {hourly_dfs[i].shape[0]}")
    hourly_dfs[i] = hourly_dfs[i].drop_duplicates(["Id", "ActivityHour"])
    print(f"After: {hourly_dfs[i].shape[0]}")
    print("*" * 80)

hourlyCalories_merged.csv
Before: 22099
After: 22099
********************************************************************************
hourlyIntensities_merged.csv
Before: 22099
After: 22099
********************************************************************************
hourlySteps_merged.csv
Before: 22099
After: 22099
********************************************************************************


### Merge Data

In [83]:
hourly = hourly_dfs[0]

for df in hourly_dfs[1:]:
    hourly = pd.merge(hourly, df, on=["Id", "ActivityHour"], how="outer")

hourly

,Id,ActivityHour,Calories,TotalIntensity,AverageIntensity,StepTotal
0,1503960366,4/12/2016 12:00:00 AM,81,20,0.333333,373
1,1503960366,4/12/2016 1:00:00 AM,61,8,0.133333,160
2,1503960366,4/12/2016 2:00:00 AM,59,7,0.116667,151
3,1503960366,4/12/2016 3:00:00 AM,47,0,0.000000,0
4,1503960366,4/12/2016 4:00:00 AM,48,0,0.000000,0
...,...,...,...,...,...,...
22094,8877689391,5/12/2016 10:00:00 AM,126,12,0.200000,514
22095,8877689391,5/12/2016 11:00:00 AM,192,29,0.483333,1407
22096,8877689391,5/12/2016 12:00:00 PM,321,93,1.550000,3135
22097,8877689391,5/12/2016 1:00:00 PM,101,6,0.100000,307


### Convert Date/Time

In [88]:
hourly.ActivityHour = pd.to_datetime(hourly.ActivityHour)

### Write CSV

In [89]:
hourly.to_csv(path_clean + "hourly_clean.csv", index=False)

---

## Minutes

### Load Data

In [175]:
minute_fnames = [f for f in os.listdir(path_raw) if "minute" in f and "Wide" not in f]

minute_dfs = {f.split("_merged")[0].split("Narrow")[0].split("minute")[-1]: pd.read_csv(path_raw + f) for f in minute_fnames}

In [176]:
minute_dfs.keys()

dict_keys(['Calories', 'Intensities', 'METs', 'Sleep', 'Steps'])

### Clean Sleep Table

In [177]:
# Rename features in sleep dataframe
minute_dfs["Sleep"] = minute_dfs["Sleep"].rename({"date": "ActivityMinute", "value": "Sleep"}, axis=1)

In [178]:
minute_dfs["Sleep"] = minute_dfs["Sleep"].drop("logId", axis=1)

### Drop Duplicates

In [179]:
for k, v in minute_dfs.items():
    print(k)
    print(f"Before: {v.shape[0]}")
    v = v.drop_duplicates(["Id", "ActivityMinute"])
    print(f"After: {v.shape[0]}")
    print("*" * 80)

Calories
Before: 1325580
After: 1325580
********************************************************************************
Intensities
Before: 1325580
After: 1325580
********************************************************************************
METs
Before: 1325580
After: 1325580
********************************************************************************
Sleep
Before: 188521
After: 187978
********************************************************************************
Steps
Before: 1325580
After: 1325580
********************************************************************************


### Convert to Datetimes

In [180]:
for k, v in minute_dfs.items():
    print(k)
    v.ActivityMinute = pd.to_datetime(v.ActivityMinute, format="%m/%d/%Y %I:%M:%S %p")

Calories
Intensities
METs
Sleep
Steps


### Round to Nearest Minute

In [182]:
for k, v in minute_dfs.items():
    print(k)
    v.ActivityMinute = v.ActivityMinute.dt.round("T")

Calories
Intensities
METs
Sleep
Steps


### Merge

In [183]:
minute_dfs["Calories"]

,Id,ActivityMinute,Calories
0,1503960366,2016-04-12 00:00:00,0.78650
1,1503960366,2016-04-12 00:01:00,0.78650
2,1503960366,2016-04-12 00:02:00,0.78650
3,1503960366,2016-04-12 00:03:00,0.78650
4,1503960366,2016-04-12 00:04:00,0.78650
...,...,...,...
1325575,8877689391,2016-05-12 13:55:00,1.33353
1325576,8877689391,2016-05-12 13:56:00,1.33353
1325577,8877689391,2016-05-12 13:57:00,1.33353
1325578,8877689391,2016-05-12 13:58:00,1.33353


In [184]:
minute = minute_dfs["Calories"].copy()

for k, df in minute_dfs.items():
    if k != "Calories":
        minute = pd.merge(minute, df, on=["Id", "ActivityMinute"], how="outer")

minute

,Id,ActivityMinute,Calories,Intensity,METs,Sleep,Steps
0,1503960366,2016-04-12 00:00:00,0.7865,0.0,10.0,NaN,0.0
1,1503960366,2016-04-12 00:01:00,0.7865,0.0,10.0,NaN,0.0
2,1503960366,2016-04-12 00:02:00,0.7865,0.0,10.0,NaN,0.0
3,1503960366,2016-04-12 00:03:00,0.7865,0.0,10.0,NaN,0.0
4,1503960366,2016-04-12 00:04:00,0.7865,0.0,10.0,NaN,0.0
...,...,...,...,...,...,...,...
1357834,8378563200,2016-04-11 23:55:00,NaN,NaN,NaN,1.0,NaN
1357835,8378563200,2016-04-11 23:56:00,NaN,NaN,NaN,1.0,NaN
1357836,8378563200,2016-04-11 23:57:00,NaN,NaN,NaN,1.0,NaN
1357837,8378563200,2016-04-11 23:58:00,NaN,NaN,NaN,1.0,NaN


### Write to CSV

In [187]:
minute.to_csv(path_clean + "minute_clean.csv", index=False)

## Weight Logs

In [188]:
weight = pd.read_csv(path_raw + "weightLogInfo_merged.csv")
weight

,Id,Date,WeightKg,WeightPounds,Fat,BMI,IsManualReport,LogId
0,1503960366,5/2/2016 11:59:59 PM,52.599998,115.963147,22.0,22.650000,True,1462233599000
1,1503960366,5/3/2016 11:59:59 PM,52.599998,115.963147,NaN,22.650000,True,1462319999000
2,1927972279,4/13/2016 1:08:52 AM,133.500000,294.317120,NaN,47.540001,False,1460509732000
3,2873212765,4/21/2016 11:59:59 PM,56.700001,125.002104,NaN,21.450001,True,1461283199000
4,2873212765,5/12/2016 11:59:59 PM,57.299999,126.324875,NaN,21.690001,True,1463097599000
...,...,...,...,...,...,...,...,...
62,8877689391,5/6/2016 6:43:35 AM,85.000000,187.392923,NaN,25.440001,False,1462517015000
63,8877689391,5/8/2016 7:35:53 AM,85.400002,188.274775,NaN,25.559999,False,1462692953000
64,8877689391,5/9/2016 6:39:44 AM,85.500000,188.495234,NaN,25.610001,False,1462775984000
65,8877689391,5/11/2016 6:51:47 AM,85.400002,188.274775,NaN,25.559999,False,1462949507000


In [189]:
weight.to_csv(path_clean + "weight_clean.csv", index=False)